# 머신러닝으로 자폐증 스크리닝

신경발달 장애의 조기 진단은 치료 효과를 높이고 헬스케어 비용을 상당히 낮출 수 있는 중요한 분야이다. 이 장에서는 지도 학습 방법을 통해서 행동 특성과 개인의 특징 등을 사용하여 자폐스펙트럼장애(ASD, Autism Spectrum Disorder)를 진단하는 프로젝트 예를 소개한다. 우리는 케라스를 사용한 신경망을 사용한다.

이 장의 순서는 다음과 같다.

-   머신러닝을 이용한 ADS 스크리닝
-   데이터셋 소개
-   데이터셋을 훈련 데이터와 테스트 데이터로 나누기 
-   신경망 구현
-   신경망 테스팅
-   드롭아웃 규제를 이용 과대적합 해결

## 머신러닝을 이용한 ADS 스크리닝

이 장에서는 주피터 노트북에서 케라스(Keras), 판다스(pandas), 사이킷-런(scikit-learn) 파이썬 라이브러리를 사용한다.

## 데이터셋 소개

이 장에서 사용할 데이터는 UCI 머신러닝 데이터 저장소에 있는 소아 자폐스펙트럼장애 스크리닝 데이터로 <https://archive.ics.uci.edu/ml/datasets/Autistic+Spectrum+Disorder+Screening+Data+for+Children++>에서 확인할 수 있다. 이 데이터셋에는 292명의 자폐증 스크린 데이터가 포함되어 있다. 주요 변수로는 나이, 인종, 자폐증 가족력 등이 있으다. 우리는 이 데이터셋을 사용하여 실제로 환자 자폐증을 가지고 있는지 예측해 볼 것이다. 

`.arff` 파일을 직접 오는 파이썬 패키지도 있지만 원서에서 소개한 대로 약간의 수작업을 통해 텍스트 파일을 만들었다. 이 파일은 소스 파일과 함께 제공되는 데, 이 장에 해당되는 폴더에 `Autism-Child-Data.txt` 파일로 준비해 놓았다.

## 필요한 라이브러와 데이터 임포트

주피터 노트북에서 이 장에서 사용한 라이브러리들과 데이터셋을 로딩한다. 데이터셋은 폴더에 `Autism-Child-Data.txt`로 준비해 두었다. 

먼저 라이러리들을 로링하고 버전을 확인한다. 

In [1]:
import sys
import pandas as pd
import sklearn
import keras

print('Python: {}'.format(sys.version) )
print('Pandas: {}'.format(pd.__version__))
print('Sklearn: {}'.format(sklearn.__version__) )
print('Keras: {}'.format(keras.__version__) )

Python: 3.8.6 | packaged by conda-forge | (default, Jan 25 2021, 23:22:12) 
[Clang 11.0.1 ]
Pandas: 1.2.4
Sklearn: 0.24.2
Keras: 2.4.3


이제 데이터셋을 로딩한다. 

In [2]:
# 데이터셋 임포트 
# csv 파일 읽기
data = pd.read_table('Autism-Child-Data.txt', sep = ',', index_col = None)
data.loc[0]

A1_Score                               1
A2_Score                               1
A3_Score                               0
A4_Score                               0
A5_Score                               1
A6_Score                               1
A7_Score                               0
A8_Score                               1
A9_Score                               0
A10_Score                              0
age_numeric                            6
gender                                 m
ethnicity                         Others
jundice                               no
family_history_of_austim              no
contry_of_res                     Jordan
used_app_before                       no
result                                 5
age_desc                    '4-11 years'
relation                          Parent
Class/ASD                             NO
Name: 0, dtype: object

## 데이터셋 탐색하기 

데이터셋을 탐색하여 어떤 정보가 담겨져 있는지 본다. 데이터프레임의 형태로 데이터의 갯수를 확인한다. 다음과 같은 코드를 사용한다. 

In [3]:
# 데이터프레임의 형태 출력
print('Shape of DataFrame: {}'.format(data.shape))
print(data.loc[0])

Shape of DataFrame: (292, 21)
A1_Score                               1
A2_Score                               1
A3_Score                               0
A4_Score                               0
A5_Score                               1
A6_Score                               1
A7_Score                               0
A8_Score                               1
A9_Score                               0
A10_Score                              0
age_numeric                            6
gender                                 m
ethnicity                         Others
jundice                               no
family_history_of_austim              no
contry_of_res                     Jordan
used_app_before                       no
result                                 5
age_desc                    '4-11 years'
relation                          Parent
Class/ASD                             NO
Name: 0, dtype: object


In [4]:
# 여러 환자의 대한 정보 출력 
data.loc[:10]

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,...,gender,ethnicity,jundice,family_history_of_austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
0,1,1,0,0,1,1,0,1,0,0,...,m,Others,no,no,Jordan,no,5,'4-11 years',Parent,NO
1,1,1,0,0,1,1,0,1,0,0,...,m,'Middle Eastern ',no,no,Jordan,no,5,'4-11 years',Parent,NO
2,1,1,0,0,0,1,1,1,0,0,...,m,?,no,no,Jordan,yes,5,'4-11 years',?,NO
3,0,1,0,0,1,1,0,0,0,1,...,f,?,yes,no,Jordan,no,4,'4-11 years',?,NO
4,1,1,1,1,1,1,1,1,1,1,...,m,Others,yes,no,'United States',no,10,'4-11 years',Parent,YES
5,0,0,1,0,1,1,0,1,0,1,...,m,?,no,yes,Egypt,no,5,'4-11 years',?,NO
6,1,0,1,1,1,1,0,1,0,1,...,m,White-European,no,no,'United Kingdom',no,7,'4-11 years',Parent,YES
7,1,1,1,1,1,1,1,1,0,0,...,f,'Middle Eastern ',no,no,Bahrain,no,8,'4-11 years',Parent,YES
8,1,1,1,1,1,1,1,0,0,0,...,f,'Middle Eastern ',no,no,Bahrain,no,7,'4-11 years',Parent,YES
9,0,0,1,1,1,0,1,1,0,0,...,f,?,no,yes,Austria,no,5,'4-11 years',?,NO


데이터프레임에 대한 전체적인 조망을 위해 `data.describe()` 함수를 적용하여 모든 숫자형 데이터에 대하여 평균, 최솟값, 최댓값 등을 확인한다.

In [5]:
# 데이터프레임에 대한 설명
data.describe()

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,result
count,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000
mean,0.633562,0.534247,0.743151,0.551370,0.743151,0.712329,0.606164,0.496575,0.493151,0.726027,6.239726
std,0.482658,0.499682,0.437646,0.498208,0.437646,0.453454,0.489438,0.500847,0.500811,0.446761,2.284882
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,6.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,10.000000


In [6]:
data.dtypes

A1_Score                     int64
A2_Score                     int64
A3_Score                     int64
A4_Score                     int64
A5_Score                     int64
A6_Score                     int64
A7_Score                     int64
A8_Score                     int64
A9_Score                     int64
A10_Score                    int64
age_numeric                 object
gender                      object
ethnicity                   object
jundice                     object
family_history_of_austim    object
contry_of_res               object
used_app_before             object
result                       int64
age_desc                    object
relation                    object
Class/ASD                   object
dtype: object

## 데이터 전처리 

데이터 전처리 과정을 살펴보자. 데이터셋에 있는 모든 정보를 필요로 하는 것은 아니다. 이 데이터셋은 모두 4세에서 11세까지의 소아를 대상으로 하고 있어서 나이 기술(`age_desc` 변수)은 필요하지 않다. 그리고 `result` 변수는 설문 방법에 따른 가중치를 계산한 것으로 여기서는 사용하지 않을 것이다. 

In [7]:
# 필요없는 열 삭제 
data = data.drop(['result', 'age_desc'], axis=1)

다음은 데이터프레임을 입력 데이터로 사용할 `x`, 타깃 데이터로 사용할 `y`로 나눈다. 여기서 `x`는 타깃이 되는 `Class/ASD`를 제외한 모든 변수가 포함되고, `y`에는 `Class/ASD` 변수만 포함된다. 다음과 같은 코드를 사용한다. 

In [8]:
x = data.drop(['Class/ASD'], 1)
y = data['Class/ASD']

다음 코드로 `x` 데이터셋을 확인한다. 

In [9]:
x.loc[:10]

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age_numeric,gender,ethnicity,jundice,family_history_of_austim,contry_of_res,used_app_before,relation
0,1,1,0,0,1,1,0,1,0,0,6,m,Others,no,no,Jordan,no,Parent
1,1,1,0,0,1,1,0,1,0,0,6,m,'Middle Eastern ',no,no,Jordan,no,Parent
2,1,1,0,0,0,1,1,1,0,0,6,m,?,no,no,Jordan,yes,?
3,0,1,0,0,1,1,0,0,0,1,5,f,?,yes,no,Jordan,no,?
4,1,1,1,1,1,1,1,1,1,1,5,m,Others,yes,no,'United States',no,Parent
5,0,0,1,0,1,1,0,1,0,1,4,m,?,no,yes,Egypt,no,?
6,1,0,1,1,1,1,0,1,0,1,5,m,White-European,no,no,'United Kingdom',no,Parent
7,1,1,1,1,1,1,1,1,0,0,5,f,'Middle Eastern ',no,no,Bahrain,no,Parent
8,1,1,1,1,1,1,1,0,0,0,11,f,'Middle Eastern ',no,no,Bahrain,no,Parent
9,0,0,1,1,1,0,1,1,0,0,11,f,?,no,yes,Austria,no,?


## 원-핫 인코딩 

In [10]:
# 카테고리형 값들을 원-핫 인코딩 벡터로 변환
X = pd.get_dummies(x)

In [11]:
# 새로 만들어진 카테고리형 열 레이블 출력
X.columns.values

array(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score',
       'A6_Score', 'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score',
       'age_numeric_10', 'age_numeric_11', 'age_numeric_4',
       'age_numeric_5', 'age_numeric_6', 'age_numeric_7', 'age_numeric_8',
       'age_numeric_9', 'age_numeric_?', 'gender_f', 'gender_m',
       "ethnicity_'Middle Eastern '", "ethnicity_'South Asian'",
       'ethnicity_?', 'ethnicity_Asian', 'ethnicity_Black',
       'ethnicity_Hispanic', 'ethnicity_Latino', 'ethnicity_Others',
       'ethnicity_Pasifika', 'ethnicity_Turkish',
       'ethnicity_White-European', 'jundice_no', 'jundice_yes',
       'family_history_of_austim_no', 'family_history_of_austim_yes',
       "contry_of_res_'Costa Rica'", "contry_of_res_'Isle of Man'",
       "contry_of_res_'New Zealand'", "contry_of_res_'Saudi Arabia'",
       "contry_of_res_'South Africa'", "contry_of_res_'South Korea'",
       "contry_of_res_'U.S. Outlying Islands'",
       "contry_of_res_'United A

이제 한 명의 환자의 데이터를 출력해 보자. 

In [12]:
# 변환된 데이터셋에서 한 환자의 데이터 보기 
X.loc[1]

A1_Score             1
A2_Score             1
A3_Score             0
A4_Score             0
A5_Score             1
                    ..
relation_?           0
relation_Parent      1
relation_Relative    0
relation_Self        0
relation_self        0
Name: 1, Length: 96, dtype: int64

In [13]:
# 클래스 데이터에 있는 카테고리형 변수를 원-핫 코딩 벡터로 변환
Y = pd.get_dummies(y)

## 데이터셋을 훈련 데이터와 테스트 데이터로 나누기 


In [14]:
from sklearn import model_selection
# 훈련 데이터와 테스트 데이터로 나누기 
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size = 0.2)

In [15]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(233, 96)
(59, 96)
(233, 2)
(59, 2)


`X` 데이터셋은 훈련 데이터에는 233개의 행, 테스트 59개의 행을 가지고 있고, 열을 모두 96개이다. `Y` 데이터에는 원래의 `Class` 레이블이 `YES`, `NO` 값을 가질 수 있는데, 원-핫 인코딩되어 2개의 열을 가진다. 

## 신경망 구현 

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [17]:
# 케라스 모델을 만들기 위한 함수 정의 
def create_model():
  # 모델 생성
  model = Sequential()
  model.add(Dense(8, input_dim=96, kernel_initializer='normal',
            activation='relu'))
  model.add(Dense(4, kernel_initializer='normal', activation='relu'))
  model.add(Dense(2, activation='sigmoid'))
  
  # 모델 컴파일
  adam = Adam(lr=0.001)
  model.compile(loss='categorical_crossentropy',
                optimizer=adam, metrics=['accuracy'])
  return model

In [18]:
model = create_model()

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 776       
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 10        
Total params: 822
Trainable params: 822
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
model.fit(X_train, Y_train, epochs=50, 
          batch_size=10, verbose = 1)

Epoch 1/50
24/24 [==============================] - 0s 811us/step - loss: 0.6907 - accuracy: 0.5768
Epoch 2/50
24/24 [==============================] - 0s 821us/step - loss: 0.6839 - accuracy: 0.5404
Epoch 3/50
24/24 [==============================] - 0s 688us/step - loss: 0.6777 - accuracy: 0.4912
Epoch 4/50
24/24 [==============================] - 0s 732us/step - loss: 0.6566 - accuracy: 0.5240
Epoch 5/50
24/24 [==============================] - 0s 789us/step - loss: 0.6346 - accuracy: 0.6727
Epoch 6/50
24/24 [==============================] - 0s 812us/step - loss: 0.5885 - accuracy: 0.7263
Epoch 7/50
24/24 [==============================] - 0s 779us/step - loss: 0.5262 - accuracy: 0.8066
Epoch 8/50
24/24 [==============================] - 0s 718us/step - loss: 0.5137 - accuracy: 0.8800
Epoch 9/50
24/24 [==============================] - 0s 782us/step - loss: 0.4600 - accuracy: 0.8901
Epoch 10/50
24/24 [==============================] - 0s 881us/step - loss: 0.4301 - accuracy: 0.8883

결과를 보면 처음에는 정확도가 낮지만 점차 높아지고, 손실은 점점 더 낮아진다. 결과적으로 최종 50 에포크를 마치면 정확도가 100%가 되었다. 그런데 이 성적은 훈련 데이터에만 그렇다는 것을 의미한다. 

## 신경망 테스팅 

In [20]:
# 분류 모델에 대한 분류 보고서 작성
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

#predictions = model.predict_classes(X_test)
predictions = np.argmax(model.predict(X_test), axis=-1)
predictions

array([0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1])

위 코드는 1 또는 0 값으로 이뤄진 배열을 출력한다.

In [21]:
print('Prediction Results for Neural Network')
print(accuracy_score(Y_test[['YES']], predictions))
print(classification_report(Y_test[['YES']], predictions))

Prediction Results for Neural Network
0.9491525423728814
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        30
           1       1.00      0.90      0.95        29

    accuracy                           0.95        59
   macro avg       0.95      0.95      0.95        59
weighted avg       0.95      0.95      0.95        59



## 드롭아웃 정규화를 사용하여 과적합 해결하기 

과적합을 줄이는 한 가지 방법이 드롭아웃 정규화이다.

In [22]:
# 필요한 패키지 임포트
from keras.layers import Dropout  # 임포트 라인에 추가 
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# 드롭아웃 비율 
dropout_rate = 0.25

# 케라스 모델을 만들기 위한 함수 정의 
def create_model():
    # 모델 생성
    model = Sequential()
    model.add(Dense(8, input_dim=96, kernel_initializer='normal',
              activation='relu'))
    model.add(Dropout(dropout_rate))        # 드롭아웃 레이어 추가 
    model.add(Dense(4, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))        # 드롭아웃 레이어 추가 
    model.add(Dense(2, activation='sigmoid'))
    
    # 모델 컴파일
    adam = Adam(lr=0.001)
    model.compile(loss='categorical_crossentropy',
                  optimizer=adam, metrics=['accuracy'])
    return model

model = create_model()

model.fit(X_train, Y_train, epochs=50, 
          batch_size=10, verbose = 1)

predictions = np.argmax(model.predict(X_test), axis=-1)


print('Prediction Results for Neural Network')
print(accuracy_score(Y_test[['YES']], predictions))
print(classification_report(Y_test[['YES']], predictions))

Epoch 1/50
24/24 [==============================] - 0s 708us/step - loss: 0.6920 - accuracy: 0.5556
Epoch 2/50
24/24 [==============================] - 0s 794us/step - loss: 0.6901 - accuracy: 0.6185
Epoch 3/50
24/24 [==============================] - 0s 769us/step - loss: 0.6853 - accuracy: 0.6572
Epoch 4/50
24/24 [==============================] - 0s 687us/step - loss: 0.6762 - accuracy: 0.6874
Epoch 5/50
24/24 [==============================] - 0s 867us/step - loss: 0.6559 - accuracy: 0.7253
Epoch 6/50
24/24 [==============================] - 0s 793us/step - loss: 0.6323 - accuracy: 0.6745
Epoch 7/50
24/24 [==============================] - 0s 805us/step - loss: 0.5870 - accuracy: 0.7870
Epoch 8/50
24/24 [==============================] - 0s 716us/step - loss: 0.5449 - accuracy: 0.7729
Epoch 9/50
24/24 [==============================] - 0s 768us/step - loss: 0.4916 - accuracy: 0.8358
Epoch 10/50
24/24 [==============================] - 0s 822us/step - loss: 0.4413 - accuracy: 0.8766

## 요약

이 장에서 머신 러닝을 사용하여 약 90% 정확도로 자폐증을 예측할 수 있었다. 이 장에서는 카테고리형 데이터를 머신 러닝에 사용할 수 있도록 가변수 처리하는 방법을 살펴보았다. 헬스케어 데이터에는 카테고리형 데이터가 많이 사용된다. 대표적인 방법은 원-핫 인코딩이다. 드롭아웃 정규화를 통해 과적합을 줄이는 방법도 설명했다.

이 책에서 다양한 헬스케어 이슈들을 머신 러닝을 적용하여 해결하는 방법을 탐구해 보았다. 1장에서는 SVM, KNN 모델을 사용하여 암세포를 탐지하는 머신 러닝을 살펴보았다. 2ㅏ장에서는 케라스를 사용한 딥러닝을 사용하여 당뇨병을 예측해 보았다. 3장에서는 흔히 사용되는 분류 모델들을 사용하여 대장균 염기서열이 프로모터인지 아닌지 예측할 수 있었다. 4장에서는 신경망을 통한 심장병 예측을 다뤘다. 마지막으로 5장에서는 자폐증을 예측하는 사례를 보았다. 이런 사례를 통해 최신의 머신 러닝 기술이 헬스케어의 다양한 질환들을 진단하고 관리하는 데 혁신을 일으킬 수 있을 것이라는 직관을 얻을 수 있었으리라 기대한다.